# Amor Reduction

## How to start:

Before starting you must:
- Have conda installed
- conda env create -f ess-notebooks-latest.yml python=3.7 . The yaml environment file is part of this repository.

## What will this notebook show?

The notebook will show how to use the `ess.amor.AmorData`, `ess.amor.AmorReference` and `ess.amor.Normalisation` classes for the reduction of data collected at the Amor instrument at PSI. 
To achieve this, we will reduce the following data files, [sample.nxs](./sample.nxs) and [reference.nxs](./references.nxs), and then normalise the sample data (with respect to the reference data). 

## Reduction

Before, we begin, we should import then necessary modules. 

In [1]:
import numpy as np
import scipp as sc
import scippneutron as scn
from ess.amor import AmorData, AmorReference, Normalisation

<IPython.core.display.Javascript object>

The `AmorData` class will take the loaded NeXus file and perform the reduction steps to obtain the reflected intensity as a function of $q_z$, including accounting for aspects such as gravity. 
The `sample_angle_offset` allows the angular offset of the sample with respect to the horizon to be accounted for.

In [3]:
sample = AmorData(scn.load_nexus('sample.nxs'), sample_angle_offset=0.04 * sc.units.deg)

Some detector and wavelength masking can then be performed.

In [4]:
sample.detector_masking(y_min=0 * sc.units.m, y_max=100e-3 * sc.units.m)
sample.wavelength_masking()

The `AmorReference` class reads the reference supermirror measurement, and will perform the necessary corrections.
For this measurement, no angular offset is required. 
Again, detector and wavelength masking is performed.

In [5]:
reference = AmorReference(scn.load_nexus('reference.nxs'))
reference.detector_masking(y_min=0 * sc.units.m, y_max=100e-3 * sc.units.m)
reference.wavelength_masking()

For the normalisation of the sample, we use the `Normalisation` class.

In [6]:
norm = Normalisation(sample, reference)

With this object, there is the choice to bin in the $\lambda$/$\theta$-space or the $q_z$-space. 

In [7]:
bins = (np.linspace(2.5, 15, 50), np.linspace(0.6, 1.25, 50))
lambda_theta = norm.wavelength_theta_bin(bins)

The data binned in $\lambda$/$\theta$-space can be investigated and plotted.

In [8]:
lambda_theta

<scipp.DataArray>
Dimensions: {{wavelength, 49}, {theta, 49}}
Coordinates:
  theta                     float64    [deg]            (theta [bin-edge])  [0.600000, 0.613265, ..., 1.236735, 1.250000]
  wavelength                float64    [Å]             (wavelength [bin-edge])  [2.500000, 2.755102, ..., 14.744898, 15.000000]
Data:
                            float32    [dimensionless]  (wavelength, theta)  [0.105753, 0.139934, ..., 1.299265, 1.678631]  [0.002249, 0.001972, ..., 0.452168, 0.439621]
Attributes:
  experiment_title          string     [dimensionless]  ()  ["commissioning"]
  instrument_name           string     [dimensionless]  ()  ["AMOR"]
  sample_position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, 0.000000)]
  source_position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, -15.000000)]

In [9]:
sc.plot(lambda_theta, norm='log')

In [10]:
q_bins = np.linspace(0.007, 0.09, 50)
q = norm.q_bin(q_bins)

As can be done for the $q_z$-binned data.

In [11]:
q

<scipp.DataArray>
Dimensions: {{qz, 49}}
Coordinates:
  qz                        float64    [1/Å]           (qz [bin-edge])  [0.007000, 0.008694, ..., 0.088306, 0.090000]
  sigma_qz_by_qz            float64    [dimensionless]  (qz)  [0.038926, 0.038712, ..., 0.029785, 0.029040]
Data:
                            float32    [dimensionless]  (qz)  [5.201439, 4.269416, ..., 0.011207, 0.004834]  [0.101002, 0.014867, ..., 0.000016, 0.000008]
Attributes:
  experiment_title          string     [dimensionless]  ()  ["commissioning"]
  instrument_name           string     [dimensionless]  ()  ["AMOR"]
  sample_position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, 0.000000)]
  source_position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, -15.000000)]

In [12]:
sc.plot(sc.log10(q))